In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.models.segmentation import deeplabv3_resnet50
from torch.utils.data import DataLoader
from dataset import CityscapesDataset

# Setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 4
num_classes = 19
epochs = 20

# Transforms
transform = transforms.Compose([
    transforms.Resize((256, 512)),
    transforms.ToTensor(),
])

# Dataset
train_dataset = CityscapesDataset(root='datasets/cityscapes', split='train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Model
model = deeplabv3_resnet50(pretrained=False, num_classes=num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss(ignore_index=255)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training loop
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, masks in train_loader:
        images, masks = images.to(device), masks.to(device)
        outputs = model(images)['out']
        loss = criterion(outputs, masks)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

# Save the model
torch.save(model.state_dict(), 'deeplabv3_cityscapes.pth')



RuntimeError: size mismatch (got input: [4, 19, 256, 512] , target: [4, 96, 256]